In [8]:
# Import required libraries
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set display options
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)

# Set plot style
plt.style.use('default')  # Using default matplotlib style
sns.set_theme(style="whitegrid")  # Add seaborn grid style


In [9]:
# Initialize connection to MetaTrader 5
def connect_mt5():
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()
        return False
    
    # Display MT5 terminal info
    print(f"MetaTrader5 package version: {mt5.__version__}")
    print(f"Terminal info: {mt5.terminal_info()}")
    print(f"Connected to account: {mt5.account_info().login}")
    return True

# Connect to MT5
if not connect_mt5():
    print("Failed to connect to MT5")


MetaTrader5 package version: 5.0.5050
Terminal info: TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=True, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=5100, maxbars=100000, codepage=0, ping_last=54036, community_balance=0.0, retransmission=0.0, company='Raw Trading Ltd', name='MetaTrader 5 IC Markets Global', language='English', path='C:\\Program Files\\MetaTrader 5 IC Markets Global', data_path='C:\\Users\\babat\\AppData\\Roaming\\MetaQuotes\\Terminal\\010E047102812FC0C18890992854220E', commondata_path='C:\\Users\\babat\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
Connected to account: 62081127


In [11]:
# Function to fetch historical trades
def get_historical_trades(days_back=30):
    from_date = datetime.now() - timedelta(days=days_back)
    to_date = datetime.now()
    
    # Get both deals and orders history
    deals = mt5.history_deals_get(from_date, to_date)
    orders = mt5.history_orders_get(from_date, to_date)
    
    if deals is None and orders is None:
        print("No trading history found")
        return None
    
    # Process deals
    if deals is not None:
        deals_df = pd.DataFrame(list(deals), columns=deals[0]._asdict().keys())
        deals_df['time'] = pd.to_datetime(deals_df['time'], unit='s')
        
        # Filter only entry and exit deals (type 0 and 1)
        deals_df = deals_df[deals_df['entry'].isin([0, 1])]
        
        # Add trade direction and entry/exit labels
        deals_df['direction'] = deals_df['type'].map({0: 'buy', 1: 'sell'})
        deals_df['deal_type'] = deals_df['entry'].map({0: 'exit', 1: 'entry'})
        
        # Calculate trade volume in standard lots
        deals_df['volume_lots'] = deals_df['volume'] / 10000
        
        # Add commission and swap to total profit
        deals_df['total_profit'] = deals_df['profit'] + deals_df['commission'] + deals_df['swap']
    else:
        deals_df = pd.DataFrame()
    
    # Process orders
    if orders is not None:
        orders_df = pd.DataFrame(list(orders), columns=orders[0]._asdict().keys())
        orders_df['time_setup'] = pd.to_datetime(orders_df['time_setup'], unit='s')
        orders_df['time_done'] = pd.to_datetime(orders_df['time_done'], unit='s')
        
        # Add order type labels
        orders_df['order_type'] = orders_df['type'].map({
            0: 'buy market', 1: 'sell market', 2: 'buy limit', 3: 'sell limit',
            4: 'buy stop', 5: 'sell stop', 6: 'buy stop limit', 7: 'sell stop limit'
        })
    else:
        orders_df = pd.DataFrame()
    
    # Store both DataFrames in a dictionary
    trading_data = {
        'deals': deals_df,
        'orders': orders_df
    }
    
    # Print summary
    if not deals_df.empty:
        print(f"Found {len(deals_df)} deals:")
        print(f"- Total profit: {deals_df['total_profit'].sum():.2f}")
        print(f"- Win rate: {(deals_df[deals_df['profit'] > 0].shape[0] / len(deals_df) * 100):.1f}%")
    if not orders_df.empty:
        print(f"Found {len(orders_df)} orders")
    
    return trading_data

# Fetch last 30 days of trading history
trading_data = get_historical_trades(30)

# Display sample of deals if available
if trading_data is not None and not trading_data['deals'].empty:
    print("\nSample of recent deals:")
    display(trading_data['deals'].head())


Found 25 deals:
- Total profit: 49999.38
- Win rate: 24.0%
Found 25 orders

Sample of recent deals:


,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id,direction,deal_type,volume_lots,total_profit
0,22738413,0,2025-06-13 12:12:08,1749816728165,2,0,0,0,0,0.00,0.00000,0.0,0.0,50000.00,0.0,,Demo Deposit,,NaN,exit,0.000000,50000.00
1,22739521,36405615,2025-06-13 12:22:21,1749817341701,0,0,0,36405615,0,0.01,1.15277,0.0,0.0,0.00,0.0,EURUSD,,,buy,exit,0.000001,0.00
2,22739543,36405636,2025-06-13 12:22:31,1749817351390,1,1,0,36405615,0,0.01,1.15274,0.0,0.0,-0.03,0.0,EURUSD,,,sell,entry,0.000001,-0.03
3,22739560,36405666,2025-06-13 12:22:36,1749817356094,0,0,0,36405666,0,0.01,1.15299,0.0,0.0,0.00,0.0,EURUSD,,,buy,exit,0.000001,0.00
4,22739561,36405667,2025-06-13 12:22:38,1749817358197,0,0,0,36405667,0,0.01,1.35504,0.0,0.0,0.00,0.0,GBPUSD,,,buy,exit,0.000001,0.00


In [15]:
# No need for this line anymore as trading_data is already correctly set

In [19]:
# Calculate trade statistics
def analyze_trades(trading_data):
    if trading_data is None or trading_data['deals'].empty:
        return None
    
    deals_df = trading_data['deals']
    
    # Calculate profit metrics using total_profit (includes commission and swap)
    profitable_deals = deals_df[deals_df['total_profit'] > 0]
    losing_deals = deals_df[deals_df['total_profit'] < 0]
    
    stats = {
        'total_trades': len(deals_df),
        'profitable_trades': len(profitable_deals),
        'losing_trades': len(losing_deals),
        'total_profit': deals_df['total_profit'].sum(),
        'total_commission': deals_df['commission'].sum(),
        'total_swap': deals_df['swap'].sum(),
        'avg_profit': deals_df['total_profit'].mean(),
        'max_profit': deals_df['total_profit'].max(),
        'max_loss': deals_df['total_profit'].min(),
        'win_rate': len(profitable_deals) / len(deals_df) * 100,
        'avg_volume': deals_df['volume_lots'].mean(),
        'total_volume': deals_df['volume_lots'].sum()
    }
    
    # Calculate profit factor (avoid division by zero)
    if len(losing_deals) > 0 and abs(losing_deals['total_profit'].sum()) > 0:
        stats['profit_factor'] = abs(profitable_deals['total_profit'].sum() / 
                                   losing_deals['total_profit'].sum())
    else:
        stats['profit_factor'] = float('inf') if len(profitable_deals) > 0 else 0
    
    # Add symbol statistics
    symbol_stats = deals_df.groupby('symbol')['total_profit'].agg([
        ('trades_per_symbol', 'count'),
        ('profit_per_symbol', 'sum')
    ])
    stats['most_traded_symbol'] = symbol_stats['trades_per_symbol'].idxmax()
    stats['most_profitable_symbol'] = symbol_stats['profit_per_symbol'].idxmax()
    
    return pd.Series(stats)

# Calculate and display statistics
if trading_data is not None and not trading_data['deals'].empty:
    stats = analyze_trades(trading_data)
    print("\nTrading Statistics:")
    display(stats)


In [20]:
# Visualize trade performance
def plot_trade_analysis(trading_data):
    if trading_data is None or trading_data['deals'].empty:
        return
    
    deals_df = trading_data['deals']
    
    # Create a figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Cumulative profit over time
    sorted_deals = deals_df.sort_values('time')
    cumulative_profit = sorted_deals['total_profit'].cumsum()
    axes[0, 0].plot(sorted_deals['time'], cumulative_profit)
    axes[0, 0].set_title('Cumulative Profit Over Time')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Profit')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Profit distribution
    axes[0, 1].hist(deals_df['total_profit'], bins=50, color='skyblue', alpha=0.7)
    axes[0, 1].axvline(x=0, color='r', linestyle='--', alpha=0.5)
    axes[0, 1].set_title('Profit Distribution')
    axes[0, 1].set_xlabel('Profit (including commission and swap)')
    axes[0, 1].set_ylabel('Frequency')
    
    # 3. Win rate by symbol
    win_rate_by_symbol = deals_df.groupby('symbol').apply(
        lambda x: (x['total_profit'] > 0).mean() * 100
    ).sort_values(ascending=False)
    
    win_rate_by_symbol.plot(kind='bar', ax=axes[1, 0], color='lightgreen', alpha=0.7)
    axes[1, 0].set_title('Win Rate by Symbol')
    axes[1, 0].set_xlabel('Symbol')
    axes[1, 0].set_ylabel('Win Rate (%)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. Average profit by hour
    deals_df['hour'] = deals_df['time'].dt.hour
    avg_profit_by_hour = deals_df.groupby('hour')['total_profit'].mean().reindex(range(24))
    
    color_map = ['red' if x < 0 else 'green' for x in avg_profit_by_hour]
    avg_profit_by_hour.plot(kind='bar', ax=axes[1, 1], color=color_map, alpha=0.7)
    axes[1, 1].set_title('Average Profit by Hour')
    axes[1, 1].set_xlabel('Hour of Day')
    axes[1, 1].set_ylabel('Average Profit')
    axes[1, 1].tick_params(axis='x', rotation=0)
    axes[1, 1].grid(True, alpha=0.3)
    
    # Add horizontal line at y=0 for profit by hour
    axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.2)
    
    plt.tight_layout()
    plt.show()
    
    # Additional visualization: Volume analysis
    plt.figure(figsize=(15, 5))
    
    # Plot volume over time
    plt.subplot(1, 2, 1)
    plt.plot(sorted_deals['time'], sorted_deals['volume_lots'], marker='o', alpha=0.5)
    plt.title('Trading Volume Over Time')
    plt.xlabel('Date')
    plt.ylabel('Volume (Lots)')
    plt.xticks(rotation=45)
    
    # Plot average volume by symbol
    plt.subplot(1, 2, 2)
    avg_volume_by_symbol = deals_df.groupby('symbol')['volume_lots'].mean().sort_values(ascending=False)
    avg_volume_by_symbol.plot(kind='bar', color='orange', alpha=0.7)
    plt.title('Average Volume by Symbol')
    plt.xlabel('Symbol')
    plt.ylabel('Average Volume (Lots)')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

# Generate visualizations
if trading_data is not None and not trading_data['deals'].empty:
    plot_trade_analysis(trading_data)
    axes[0, 1].set_title('Profit Distribution')
    axes[0, 1].set_xlabel('Profit')
    axes[0, 1].set_ylabel('Frequency')
    
    # 3. Win rate by symbol
    win_rate_by_symbol = trades_df.groupby('symbol').apply(
        lambda x: (x['profit'] > 0).mean() * 100
    ).sort_values(ascending=False)
    win_rate_by_symbol.plot(kind='bar', ax=axes[1, 0])
    axes[1, 0].set_title('Win Rate by Symbol')
    axes[1, 0].set_xlabel('Symbol')
    axes[1, 0].set_ylabel('Win Rate (%)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. Average profit by hour
    trades_df['hour'] = trades_df['time'].dt.hour
    avg_profit_by_hour = trades_df.groupby('hour')['profit'].mean()
    avg_profit_by_hour.plot(kind='bar', ax=axes[1, 1])
    axes[1, 1].set_title('Average Profit by Hour')
    axes[1, 1].set_xlabel('Hour')
    axes[1, 1].set_ylabel('Average Profit')
    axes[1, 1].tick_params(axis='x', rotation=0)
    
    plt.tight_layout()
    plt.show()

# Generate visualizations
if trades_df is not None:
    plot_trade_analysis(trades_df)


In [21]:
# Advanced Analysis: Risk Metrics
def calculate_risk_metrics(trades_df):
    if trades_df is None or len(trades_df) == 0:
        return None
    
    # Calculate daily returns
    trades_df['date'] = trades_df['time'].dt.date
    daily_profits = trades_df.groupby('date')['profit'].sum()
    
    metrics = {
        'sharpe_ratio': daily_profits.mean() / daily_profits.std() * np.sqrt(252),  # Annualized
        'max_drawdown': (daily_profits.cumsum() - daily_profits.cumsum().expanding().max()).min(),
        'profit_std': daily_profits.std(),
        'profit_skew': daily_profits.skew(),
        'profit_kurtosis': daily_profits.kurtosis(),
        'consecutive_wins': max(len(list(g)) for k, g in itertools.groupby(trades_df['profit'] > 0) if k),
        'consecutive_losses': max(len(list(g)) for k, g in itertools.groupby(trades_df['profit'] < 0) if k)
    }
    
    return pd.Series(metrics)

# Calculate and display risk metrics
if trades_df is not None:
    import itertools
    risk_metrics = calculate_risk_metrics(trades_df)
    display(risk_metrics)


In [22]:
if trades_df is not None:
    import itertools
    risk_metrics = calculate_risk_metrics(trades_df)
    display(risk_metrics)